# LSTM con embeddings

Modelo básico con los embeddings de fasttext

In [1]:
%ls ../data/dev_es/

ls: cannot access '../data/dev_es/': No such file or directory


In [2]:
import pandas as pd
df_train = pd.read_table("../../../data/es/train_es.tsv", index_col="id")
df_dev = pd.read_table("../../../data/es/dev_es.tsv", index_col="id")

text_train, y_train = df_train["text"], df_train["HS"]
text_dev, y_dev = df_dev["text"], df_dev["HS"]

Tengo que hacer dos cosas:

- Primero, convertir los tweets a secuencias de texto
- Luego, paddear las secuencias a cierta longitud (Keras necesita esto para poder paralelizar cálculo)

In [3]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

num_words = 200000

tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(text_train)

X_train = tokenizer.texts_to_sequences(text_train)
X_dev = tokenizer.texts_to_sequences(text_dev)

max_length = 30

X_train = pad_sequences(X_train, max_length)
X_dev = pad_sequences(X_dev, max_length)

Using TensorFlow backend.


Carguemos embeddings

In [4]:
import os
from glob import glob

path_to_embeddings = os.path.expanduser("/home/jmperez/WordVectors/")

print("Available embeddings: ", glob(os.path.join(path_to_embeddings, "*.vec")))

Available embeddings:  ['/home/jmperez/WordVectors/UBA_w5_200.vec', '/home/jmperez/WordVectors/wiki.es.vec', '/home/jmperez/WordVectors/UBA_w5_300.vec']


# Twitter Embeddings

In [5]:
import numpy as np

word_to_vec = {}

with open(os.path.join(path_to_embeddings, "UBA_w5_300.vec")) as f:
    for line in f:
        values = line.split()
        word = values[0]
        try:
            vec = np.asarray(values[1:], dtype="float32")
        except:
            print(("*" * 80  + "\n")*3)
            print("Problema con la sig línea:")
            print(values[:10])
            word = values[1]
            vec = np.asarray(values[2:], dtype="float32")
        word_to_vec[word] = vec
        
embedding_size = len(word_to_vec["hola"])

********************************************************************************
********************************************************************************
********************************************************************************

Problema con la sig línea:
['.', '.', '-0.22232', '0.0052569', '0.47066', '0.13836', '0.15991', '0.19504', '0.00067885', '0.020299']
********************************************************************************
********************************************************************************
********************************************************************************

Problema con la sig línea:
['.', '...', '-0.11666', '-0.083768', '0.028919', '0.29973', '0.21017', '0.27808', '0.063251', '0.090223']
********************************************************************************
********************************************************************************
********************************************************************************

P

In [6]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((num_words, embedding_size))
for word, i in tokenizer.word_index.items():
    embedding_vector = word_to_vec.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [7]:
embedding_matrix.shape

(200000, 300)

In [8]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Dropout, Conv1D, Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence


model = Sequential()
model.add(Embedding(num_words, embedding_size, input_length=max_length, 
                    weights=[embedding_matrix], trainable=False))
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.5))
model.add(Conv1D(64, 3, activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [9]:
model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=10, batch_size=32)


Train on 4469 samples, validate on 500 samples
Epoch 1/10
4469/4469 [==============================] - 8s 2ms/step - loss: 0.6349 - acc: 0.6449 - val_loss: 0.5699 - val_acc: 0.7100
Epoch 2/10
4469/4469 [==============================] - 7s 2ms/step - loss: 0.5316 - acc: 0.7342 - val_loss: 0.5439 - val_acc: 0.7300
Epoch 3/10
4469/4469 [==============================] - 7s 2ms/step - loss: 0.4595 - acc: 0.7874 - val_loss: 0.5075 - val_acc: 0.7400
Epoch 4/10
4469/4469 [==============================] - 7s 2ms/step - loss: 0.4123 - acc: 0.8085 - val_loss: 0.5116 - val_acc: 0.7800
Epoch 5/10
4469/4469 [==============================] - 7s 2ms/step - loss: 0.3866 - acc: 0.8328 - val_loss: 0.4874 - val_acc: 0.7640
Epoch 6/10
4469/4469 [==============================] - 7s 2ms/step - loss: 0.3281 - acc: 0.8575 - val_loss: 0.5325 - val_acc: 0.7740
Epoch 7/10
4469/4469 [==============================] - 7s 2ms/step - loss: 0.2789 - acc: 0.8870 - val_loss: 0.5408 - val_acc: 0.7840
Epoch 8/10
4469

In [10]:
from hate.utils import print_evaluation

print_evaluation(model, X_dev, y_dev.values)

500/500 [==============================] - 0s 623us/step
Loss        : 0.7420
Accuracy    : 0.7700
Precision   : 0.7831
Recall      : 0.6667
F1          : 0.7202


## Bidirectional LSTM

In [11]:
from keras.layers import Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Dropout, Conv1D, Flatten, Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence


model = Sequential()
model.add(Embedding(num_words, embedding_size, input_length=max_length, 
                    weights=[embedding_matrix], trainable=False))
model.add(Bidirectional(LSTM(100, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Conv1D(64, 3, activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [12]:
model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=10, batch_size=32)


Train on 4469 samples, validate on 500 samples
Epoch 1/10
4469/4469 [==============================] - 13s 3ms/step - loss: 0.6197 - acc: 0.6632 - val_loss: 0.5475 - val_acc: 0.7200
Epoch 2/10
4469/4469 [==============================] - 12s 3ms/step - loss: 0.5058 - acc: 0.7588 - val_loss: 0.5986 - val_acc: 0.6680
Epoch 3/10
4469/4469 [==============================] - 12s 3ms/step - loss: 0.4548 - acc: 0.7870 - val_loss: 0.5036 - val_acc: 0.7400
Epoch 4/10
4469/4469 [==============================] - 12s 3ms/step - loss: 0.4001 - acc: 0.8241 - val_loss: 0.4640 - val_acc: 0.7660
Epoch 5/10
4469/4469 [==============================] - 12s 3ms/step - loss: 0.3556 - acc: 0.8434 - val_loss: 0.4233 - val_acc: 0.7960
Epoch 6/10
4469/4469 [==============================] - 12s 3ms/step - loss: 0.3197 - acc: 0.8633 - val_loss: 0.5232 - val_acc: 0.7580
Epoch 7/10
4469/4469 [==============================] - 12s 3ms/step - loss: 0.2751 - acc: 0.8787 - val_loss: 0.4584 - val_acc: 0.8000
Epoch 8/

In [13]:
from hate.utils import print_evaluation

print_evaluation(model, X_dev, y_dev.values)

500/500 [==============================] - 1s 1ms/step
Loss        : 0.6086
Accuracy    : 0.8020
Precision   : 0.7552
Recall      : 0.8198
F1          : 0.7862


## Probemos GRU + Conv

In [14]:
from keras.layers import GRU

model = Sequential()
model.add(Embedding(num_words, embedding_size, input_length=max_length, 
                    weights=[embedding_matrix], trainable=False))
model.add(GRU(100, return_sequences=True))
model.add(Dropout(0.5))
model.add(Conv1D(64, 3, activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.15))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=20, batch_size=32)


Train on 4469 samples, validate on 500 samples
Epoch 1/20
4469/4469 [==============================] - 7s 2ms/step - loss: 0.6341 - acc: 0.6444 - val_loss: 0.5989 - val_acc: 0.6540
Epoch 2/20
4469/4469 [==============================] - 6s 1ms/step - loss: 0.5250 - acc: 0.7375 - val_loss: 0.5991 - val_acc: 0.6820
Epoch 3/20
4469/4469 [==============================] - 6s 1ms/step - loss: 0.4605 - acc: 0.7809 - val_loss: 0.6370 - val_acc: 0.6820
Epoch 4/20
4469/4469 [==============================] - 6s 1ms/step - loss: 0.4063 - acc: 0.8156 - val_loss: 0.5146 - val_acc: 0.7800
Epoch 5/20
4469/4469 [==============================] - 6s 1ms/step - loss: 0.3305 - acc: 0.8568 - val_loss: 0.4906 - val_acc: 0.7840
Epoch 6/20
4469/4469 [==============================] - 6s 1ms/step - loss: 0.2764 - acc: 0.8850 - val_loss: 0.5166 - val_acc: 0.7820
Epoch 7/20
4469/4469 [==============================] - 6s 1ms/step - loss: 0.2351 - acc: 0.9047 - val_loss: 0.5812 - val_acc: 0.7720
Epoch 8/20
4469

In [15]:
from hate.utils import print_evaluation

print_evaluation(model, X_dev, y_dev.values)

500/500 [==============================] - 0s 501us/step
Loss        : 1.3919
Accuracy    : 0.7680
Precision   : 0.7760
Recall      : 0.6712
F1          : 0.7198


## ¿Y si probamos Conv+GRU?

In [19]:
from keras.layers import GRU, MaxPooling1D

model = Sequential()
model.add(Embedding(num_words, embedding_size, input_length=max_length, 
                    weights=[embedding_matrix], trainable=False))
model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(GRU(100, return_sequences=False))
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=20, batch_size=32)


Train on 4469 samples, validate on 500 samples
Epoch 1/20
4469/4469 [==============================] - 5s 1ms/step - loss: 0.6373 - acc: 0.6435 - val_loss: 0.5859 - val_acc: 0.6960
Epoch 2/20
4469/4469 [==============================] - 4s 840us/step - loss: 0.5000 - acc: 0.7628 - val_loss: 0.5336 - val_acc: 0.7300
Epoch 3/20
4469/4469 [==============================] - 4s 863us/step - loss: 0.3874 - acc: 0.8360 - val_loss: 0.4763 - val_acc: 0.8020
Epoch 4/20
4469/4469 [==============================] - 4s 863us/step - loss: 0.2824 - acc: 0.8859 - val_loss: 0.5421 - val_acc: 0.7800
Epoch 5/20
4469/4469 [==============================] - 4s 835us/step - loss: 0.1955 - acc: 0.9268 - val_loss: 0.5875 - val_acc: 0.7460
Epoch 6/20
4469/4469 [==============================] - 4s 795us/step - loss: 0.1326 - acc: 0.9550 - val_loss: 0.6413 - val_acc: 0.7680
Epoch 7/20
4469/4469 [==============================] - 4s 838us/step - loss: 0.0822 - acc: 0.9734 - val_loss: 0.9684 - val_acc: 0.7100
Epo

In [20]:
from hate.utils import print_evaluation

print_evaluation(model, X_dev, y_dev.values)

500/500 [==============================] - 0s 307us/step
Loss        : 1.5117
Accuracy    : 0.7600
Precision   : 0.6796
Recall      : 0.8694
F1          : 0.7628
